In [1]:
import pandas as pd
import numpy as np
import time

today = time.strftime('%Y%m%d')

In [2]:
# bring in zoningmods fields from FBP as place holders

# read these fields in s24
lookup_fbp = pd.read_csv(r'C:\Users\blu\Documents\GitHub\bayarea_urbansim\data\zoning_mods_24.csv',
                         usecols = ['fbpzoningmodcat', 'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                    'dua_down', 'far_down', 'subsidy', 'notes', 'res_rent_cat', 'job_out_cat'])

print('zoning_mods_24 has {} unique fbpzoningmodcat'.format(lookup_fbp.shape[0]))
display(lookup_fbp.head())
#print(list(lookup_fbp))
print('dua_up has the following values: {}'.format(list(lookup_fbp.dua_up.unique())))
print('dua_down has the following values: {}'.format(list(lookup_fbp.dua_down.unique())))
print('far_up has the following values: {}'.format(list(lookup_fbp.far_up.unique())))
print('far_down has the following values: {}'.format(list(lookup_fbp.far_down.unique())))
print('add_bldg has the following values: {}'.format(list(lookup_fbp.add_bldg.unique())))
print('drop_bldg has the following values: {}'.format(list(lookup_fbp.drop_bldg.unique())))

zoning_mods_24 has 2970 unique fbpzoningmodcat


,fbpzoningmodcat,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,alamedaGGNADISNAinNA,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alamedaGGNADISNAinsfd,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alamedaGGNAHRADISNAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alamedaGGNAHRADISNAinsfd,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alamedaGGNAHRANAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


dua_up has the following values: [25.0, 35.0, 75.0, 50.0, nan, 100.0, 200.0, 150.0, 2.75, 1.5]
dua_down has the following values: [nan, 0.0]
far_up has the following values: [nan, 2.0, 9.0, 3.0]
far_down has the following values: [nan, 0.0]
add_bldg has the following values: [nan, 'HM', 'IW', 'HS']
drop_bldg has the following values: [nan, 'HM']


In [3]:
# read parcel-level EIR zoningmods master file
p10_pba50_EIR_attr = pd.read_csv('C:\\Users\\blu\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_EIR_attr_20210224.csv')
p10_pba50_EIR_attr_modcat = p10_pba50_EIR_attr.merge(lookup_fbp, 
                                                     left_on='fbpzoningm',
                                                     right_on='fbpzoningmodcat', how='left')
print('p10_pba50_EIR_attr_modcat has {} rows'.format(p10_pba50_EIR_attr_modcat.shape[0]))

C:\Users\blu\Apps\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (23,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


p10_pba50_EIR_attr_modcat has 1956208 rows


In [5]:
# collapsed to lookup table based on 'eirzoningm' and EIR geography fields, 'fbpzoningmodcat'
#      was kept to inherent Final Blueprint values 

EIR_modcat_df = p10_pba50_EIR_attr_modcat[['ACRES', 'fbpzoningmodcat', 'eirzoningm', 'juris',
                                           'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                                           'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                                           'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                           'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_df = EIR_modcat_df[['eirzoningm', 'juris',
                               'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                               'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                               'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                               'dua_down', 'far_down', 'subsidy','res_rent_cat', 'job_out_cat']].drop_duplicates()

# rename columns
EIR_modcat_df.rename(columns = {'eir_gg_id': 'gg_id',
                                'eir_tra_id': 'tra_id',
                                'eir_sesit_': 'sesit_id',
                                'eir_coc_id': 'coc_id',
                                'eir_ppa_id': 'ppa_id',
                                'eir_exp202': 'exp2020_id'}, inplace=True)

# add 'manual_county' column
juris_county = pd.read_csv(r'C:\Users\blu\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                           usecols = ['juris_name_full', 'county_id'])
juris_county.columns = ['juris','manual_county']

EIR_modcat_df = EIR_modcat_df.merge(juris_county, on='juris', how='left')

## create zoning_mods lookup table for Alt2 (repeat steps above)

### Major changes in EIR Alt2 H3 strategy:
#### 1. allow residential development in GGs for res and non_res parcels
#### 2. change upzoning levels for different jurisdiction categories
#### 3. don't allow upzoning for res parcels in CoCs

In [6]:
#list of job Job-Rich & High-Resource Juris and Adjacent Juris
jlist = ['menlo_park', 'palo_alto', 'cupertino', 'milpitas',
         'atherton', 'belmont', 'campbell', 'east_palo_alto',
         'fremont', 'hayward', 'los_altos', 'los_altos_hills', 
         'los_gatos', 'monte_sereno', 'mountain_view', 'newark', 
         'redwood_city', 'portola_valley', 'san_carlos', 'san_jose', 
         'santa_clara', 'saratoga', 'sunnyvale', 'union_city', 'woodside']

In [7]:
EIR_modcat_alt2 = EIR_modcat_df.copy()

# first, set to nan
EIR_modcat_alt2.dua_up = np.nan
EIR_modcat_alt2.far_up = np.nan
EIR_modcat_alt2.add_bldg = np.nan

# create an HRA list
hra_list = ['HRA','HRADIS']

# update values for Residential zoning change

#tra1
## no difference among juris categories in tra1
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id=='tra1'), 'add_bldg'] = 'HM'
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id=='tra1'), 'dua_up'] = 125

#tra2
## first add HM to all parcels in tra2
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)), 'add_bldg'] = 'HM'
## HRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)) & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'dua_up'] = 75

## adjust for juris in the list
### note: the following code doesn't differentiate HRA or nonHRA, because in the next step
### nonHRA gets lower upzoning, so the nonHRA in this juris list would get
### revised from 100 to 55 too
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)) & (
                    EIR_modcat_alt2.juris.isin(jlist)), 'dua_up'] = 100

## nonHRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)) & (
                    (EIR_modcat_alt2.sesit_id=='DIS') | (
                     EIR_modcat_alt2.sesit_id.isnull())), 'dua_up'] = 55

#tra3
## first add HM to all parcels in tra2
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id=='tra3'), 'add_bldg'] = 'HM'
## HRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id == 'tra3') & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'dua_up'] = 50

## adjust for juris in the list
### note: the following code doesn't differentiate HRA or nonHRA, because in the next step
### nonHRA gets lower upzoning, so the nonHRA in this juris list would get
### revised from 75 to 50 too
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id == 'tra3') & (
                     EIR_modcat_alt2.juris.isin(jlist)), 'dua_up'] = 75

## nonHRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id == 'tra3') & (
                    (EIR_modcat_alt2.sesit_id=='DIS') | (
                     EIR_modcat_alt2.sesit_id.isnull())), 'dua_up'] = 35

# non-TRA
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'add_bldg'] = 'HM'
## HRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'dua_up'] = 50

## adjust for juris in the list
### note: the following code doesn't differentiate HRA or nonHRA, because in the next step
### nonHRA gets lower upzoning, so the nonHRA in this juris list would get
### revised from 75 to 50 too
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                     EIR_modcat_alt2.juris.isin(jlist)), 'dua_up'] = 75

## nonHRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                    (EIR_modcat_alt2.sesit_id=='DIS') | (
                     EIR_modcat_alt2.sesit_id.isnull())), 'dua_up'] = 35

In [8]:
#Unincorporated w/in UGB upzoning
EIR_modcat_alt2.loc[EIR_modcat_alt2.exp2020_id == 'inun', 'dua_up'] = 2
EIR_modcat_alt2.loc[EIR_modcat_alt2.exp2020_id == 'inun', 'add_bldg'] = 'HS'

In [9]:
# adjustment for PPA
#1) dua_up and add_bldg = HM  doesn't apply PPAs; 
#2) all parcels within PPAs have drop_bldg = HM
EIR_modcat_alt2.loc[EIR_modcat_alt2.ppa_id=='ppa', 'dua_up'] = np.nan
EIR_modcat_alt2.loc[EIR_modcat_alt2.ppa_id=='ppa', 'add_bldg'] = np.nan
EIR_modcat_alt2.loc[EIR_modcat_alt2.ppa_id=='ppa', 'drop_bldg'] = 'HM'

# Then maintain PPA zoning changes in FBP
# for now remove this one below to make sure housing gets built in core area first given the lower upzoning
# EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (EIR_modcat_alt2.tra_id=='tra1'), 'far_up'] = 9 

EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.ppa_id=='ppa') & (
                     EIR_modcat_alt2.tra_id != 'tra1'), 'far_up'] = 2

EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.ppa_id=='ppa') & (
                     EIR_modcat_alt2.tra_id != 'tra1'), 'add_bldg'] = 'IW'

In [10]:
# limit development in CoCs
EIR_modcat_alt2.loc[(EIR_modcat_alt2.coc_id=='CoC') & (
                     EIR_modcat_alt2.ex_res_bldg =='res'), 'dua_up'] = np.nan
EIR_modcat_alt2.loc[(EIR_modcat_alt2.coc_id=='CoC') & (
                     EIR_modcat_alt2.ex_res_bldg =='res'), 'add_bldg'] = np.nan

In [11]:
# drop duplicates
EIR_modcat_alt2 = EIR_modcat_alt2.drop_duplicates()
print('EIR_modcat_alt2 has {} rows'.format(EIR_modcat_alt2.shape[0]))

# add 'FREQUENCE', 'SUM_ACRES' columns
EIR_modcat_stats = p10_pba50_EIR_attr_modcat.groupby('eirzoningm').agg({'ACRES': ['count','sum']}).reset_index()
EIR_modcat_stats.columns = ['eirzoningm', 'FREQUENCY', 'SUM_ACRES']
print('EIR_modcat_stats has {} rows'.format(EIR_modcat_stats.shape[0]))

EIR_modcat_alt2 = EIR_modcat_alt2.merge(EIR_modcat_stats, on='eirzoningm', how='left')
print('p10_pba50_EIR_modcat_df has {} rows'.format(EIR_modcat_alt2.shape[0]))

# add 'modcat_id' column
EIR_modcat_alt2['modcat_id'] = EIR_modcat_alt2.index + 1

# recoder the fields
EIR_modcat_alt2 = EIR_modcat_alt2[['eirzoningm', 'modcat_id', 'FREQUENCY', 'SUM_ACRES', 'manual_county', 'juris', 
                                   'gg_id', 'tra_id', 'sesit_id', 'coc_id', 'ppa_id', 'exp2020_id', 'ex_res_bldg', 
                                   'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_alt2 has 3535 rows
EIR_modcat_stats has 3535 rows
p10_pba50_EIR_modcat_df has 3535 rows


In [13]:
#check
# PPA parcels should have no dua_up
ppa_chk = EIR_modcat_alt2.loc[EIR_modcat_alt2.ppa_id == 'ppa']
display(ppa_chk.dua_up.unique())  # should only contain nan

# PPA parcels should have drop_bldg = HM
display(ppa_chk.drop_bldg.unique())  # should only contain 'HM'

array([ nan])

array(['HM'], dtype=object)

In [14]:
# export
EIR_modcat_alt2.rename(columns={'eirzoningm': 'eirzoningmodcat'}, inplace=True)
print('export zoning_mods lookup table of {} rows'.format(EIR_modcat_alt2.shape[0]))
EIR_modcat_alt2.to_csv('C:\\Users\\blu\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\BAUS input files\\zoning_mods_28_{}.csv'.format(today), index=False)

export zoning_mods lookup table of 3535 rows
